In [2]:
import pandas as pd
import numpy as np

wdata = open('datasets/web_log_data.csv','r').readlines()

print('\n'.join(wdata[:3]))

ip,date_time,request,step,session,user_id

c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3

visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12



In [60]:
# set names of pandas dataframe
names=['Host','Datetime', 'Request', 'Step', 'Session', 'User_ID']
# read the dataframe
df = pd.read_csv('datasets/web_log_data.csv', sep=',', names=names, header=None)

In [61]:
df.head()

,Host,Datetime,Request,Step,Session,User_ID
0,ip,date_time,request,step,session,user_id
1,c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3
2,visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12
3,dsl-61-95-54-84.requ,19/Apr/2005:08:33:01,/,1,13,13
4,d220-236-91-52.dsl.n,19/Apr/2005:09:16:06,/,1,15,15


In [50]:
def extract_method_and_protocol(row):
    # function to extract HTTP request method and protocol from a request string
    request_splits = row['Request'].split()  # split request string by space
    row['Method'] = request_splits[0]
    row['Protocol'] = request_splits[-1]
    row['Request'] = ' '.join(request_splits[1:-1])  # stitch remaining request string back
    return row

df = df.apply(extract_method_and_protocol, axis=1)

# show the result
df.head()

,Host,Date and time,Request,Step,Session,User_ID,Method,Protocol
0,ip,date_time,,step,session,user_id,request,request
1,c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,,1,3,3,/,/
2,visp.inabox.telstra.,19/Apr/2005:08:24:28,,1,12,12,/,/
3,dsl-61-95-54-84.requ,19/Apr/2005:08:33:01,,1,13,13,/,/
4,d220-236-91-52.dsl.n,19/Apr/2005:09:16:06,,1,15,15,/,/


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5867 entries, 0 to 5866
Data columns (total 8 columns):
Host             5867 non-null object
Date and time    5867 non-null object
Request          5867 non-null object
Step             5867 non-null object
Session          5867 non-null object
User_ID          5867 non-null object
Method           5867 non-null object
Protocol         5867 non-null object
dtypes: object(8)
memory usage: 366.8+ KB


In [64]:
mask = (df['Request'].str.endswith('.gif') | df['Request'].str.endswith('.jpg') | df['Request'].str.endswith('.jpeg'))
print("# Rows before:", len(df))

# invert the mask, only keep records without .gif, .jpg and .jpeg in the request column
df2 = df[~mask]

print("After images removal", len(df2))


# Rows before: 5867
After images removal 5867


In [79]:
from collections import defaultdict
import datetime

# first, make a copy of df2 just in case
df3 = df2.copy()

# sort the rows based on datetime, descending
df3.sort_values(by='Datetime', inplace=True)

# initiate session ID and user ID to 0
session_id = 0
user_id = 0

# create a dictionaries to hold last access information
last_access = defaultdict(lambda:datetime.datetime.utcfromtimestamp(0))

# dictionary to find previous session, user ID and steps assigned to a specific date/ip/browser key
session_dict = defaultdict(lambda:1)
user_id_dict = defaultdict(lambda:1)
session_steps = defaultdict(lambda:1)

In [91]:

# function to be applied row wise
# for each row, produce session, user ID and path traversal
def get_log_user_info(row):
    # access global variables shared between all rows
    global session_id, user_id, session_dict, user_id_dict, session_steps, last_access
    
    session_key = str(row['Datetime'].date()) + '_' + row['Host']  # date + IP key for finding session
    time_diff_session = row['Datetime'] - last_access[session_key]  # session time diff
    time_diff_user = row['Datetime'] - last_access[user_key]  # user time diff
    
    
    # if the time diff from previous session is > 60 mins, assign new user ID
    if time_diff_user.total_seconds() > 3600:
        user_id += 1
        user_id_dict[user_key] = user_id
        
    # update last access for session and user
    last_access[session_key] = row['Datetime']
    last_access[user_key] = row['Datetime']
    
    # assign extracted info from the row
    row['Session'] = session_dict[session_key]
    row['Step'] = session_steps[row['Session']]
    row['User_ID'] = user_id_dict[user_key]
    session_steps[row['Session']] += 1
    return row
    
# apply function above to get a new dataframe with added information


In [92]:
df3 = df3.apply(get_log_user_info, axis=1)

AttributeError: ("'str' object has no attribute 'date'", 'occurred at index 178')